# OBTENER LISTADO DE LICITACIONES DIARIAS

http://www.mercadopublico.cl/fichaLicitacion.html?idLicitacion=5546-3-LP23 ---> Ficha de licitaciones

In [1]:
import pandas as pd
from pandas import json_normalize
from datetime import datetime, timedelta
import requests
import json
from os import wait
import time



In [2]:
  global MydirData

  MydirData= "/content/drive/MyDrive/UDD_Data_Science/CasoUso/data/"

In [3]:
# Parto cargando my Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)  # si es que ya lo habia montado, fuerzo el punto de montaje



Mounted at /content/drive


In [19]:
def generar_lista_fechas(dia_init, mes_init, year_init,  dia_fin=0, mes_fin=0, year_fin=0):
    # Definir fecha de inicio y fecha actual
    fecha_inicio = datetime(year_init, mes_init, dia_init)
    if year_fin > 0:
        fecha_fin = datetime(year_fin, mes_fin, dia_fin)
    else:
        fecha_fin = datetime.today()

    # Generar lista de fechas
    lista_fechas = []
    while fecha_inicio <= fecha_fin:
        lista_fechas.append(fecha_inicio.strftime('%d%m%Y'))
        fecha_inicio += timedelta(days=1)

    return lista_fechas

def generar_url_licitaciones(fechas):
    url_base = "https://api.mercadopublico.cl/servicios/v1/publico/licitaciones.json?fecha="
    urls = []
    for fecha in fechas:
        url = url_base + fecha + "&ticket=8758C2C5-3863-4889-B05B-2D7195E78079"
        urls.append(url)
    return urls


def get_lista_licitaciones(url):
    # print("get_lista_licitaciones", url)
    # Obtener el contenido desde la URL
    response = requests.get(url)

    # Verificar si la solicitud fue exitosa
    if response.status_code != 200:
        print(f"Error al obtener datos: {response.status_code}")
        return None
        exit
    else:
         # Instead of directly calling response.json(), try to decode the content manually:
        try:
             datos = json.loads(response.content.decode('utf-8'))  # Decode using UTF-8
             df = pd.DataFrame(datos)
             return df
             #print(df.head(5))
        except json.JSONDecodeError as e:
             print(f"Error decoding JSON: {e}")
             print("Response content:", response.content)  # Print the raw content for inspection
             return None

def get_descripcion_licitacion(url_detalle_descripcion):

    print("---GetDescripcion-----")
    response = requests.get(url_detalle_descripcion)
    print("---response-----", response)
    if response.status_code != 200:
        print(f"Error al obtener datos: {response.status_code}")
        print("------>", url_detalle_descripcion)
        return None, None, None, None, None, None, None, None, None, None
        #exit
    else:
         # Instead of directly calling response.json(), try to decode the content manually:
        try:
           licitacion = json.loads(response.content.decode('utf-8'))  # Decode using UTF-8
           l2 = json_normalize(licitacion['Listado'])
           #print(l2["Items.Listado"][0])

            # Check if 'Items.Listado' exists in l2 before accessing it
           if 'Items.Listado' in l2 and l2["Items.Listado"][0] is not None:  # Check if key exists and is not None
               items = json_normalize(l2["Items.Listado"][0])
               #print("comprador ==>", l2["Comprador.CodigoOrganismo"][0])

               return ( l2["Descripcion"][0], items["CodigoCategoria"][0], items["Categoria"][0],
                        items["CodigoProducto"][0], items["NombreProducto"][0],
                        l2["Comprador.CodigoOrganismo"][0], l2["Comprador.NombreOrganismo"][0], l2["Comprador.RutUnidad"][0],
                        l2["Comprador.ComunaUnidad"][0],l2["Comprador.RegionUnidad"][0])
           else:
               return None, None, None, None, None, None, None, None, None, None

        except json.JSONDecodeError as e:
             print(f"Error decoding JSON: {e}")
             print("Response content:", response.content)  # Print the raw content for inspection
             print("------>", url_detalle_descripcion)
             return None, None, None, None, None, None, None, None, None, None


In [20]:
def completar_detalle(df_detalle_licitaciones, stop=-1, i=0):
   url_detalle_base = "https://api.mercadopublico.cl/servicios/v1/publico/licitaciones.json?codigo="
   url_detalle_fin = "&ticket=8758C2C5-3863-4889-B05B-2D7195E78079"

   df_aux=df_detalle_licitaciones.iloc[i:]
   print("stop:", stop, "   i=",i)
   for id_licitacion in df_aux['CodigoExterno']:
       print(i, " - ", id_licitacion)
       url_detalle = url_detalle_base + str(id_licitacion) + url_detalle_fin
       print(url_detalle)
       (descripcion, codigocategoria, categoria, codigoproducto, producto, idorganismo, organismo, rutunidad, comuna, region )= get_descripcion_licitacion(url_detalle)
       print(descripcion)

       df_detalle_licitaciones.at[i, 'Descripcion'] = descripcion
       df_detalle_licitaciones.at[i, 'CodigoCategoria'] = codigocategoria
       df_detalle_licitaciones.at[i, 'Categoria'] = categoria
       df_detalle_licitaciones.at[i, 'CodigoProducto'] = codigoproducto
       df_detalle_licitaciones.at[i, 'NombreProducto'] = producto

       df_detalle_licitaciones.at[i, "CodigoOrganismo"] = idorganismo
       df_detalle_licitaciones.at[i, "NombreOrganismo"] = organismo
       df_detalle_licitaciones.at[i, "RutUnidad"] = rutunidad
       df_detalle_licitaciones.at[i, "ComunaUnidad"] = comuna
       df_detalle_licitaciones.at[i, "RegionUnidad"] = region

       time.sleep(1.5)

       if i % 10 == 0:
          print("file ",i)
          myfile = MydirData + 'df_licitaciones' + str(i) + '.csv'
          df_detalle_licitaciones.to_csv(myfile, index=False)

       if stop > 0 and i >= stop:
          break
       i=i+1

   i=i+1
   print("file ",i)
   myfile = MydirData + 'licitaciones' + str(i) + '.csv'
   df_detalle_licitaciones.to_csv(myfile, index=False)
   return df_detalle_licitaciones




In [6]:
# Llamar la función
lista_fechas = generar_lista_fechas(1,1,2025,  3,5,2025)
df_urls_licitaciones = generar_url_licitaciones(lista_fechas)

#for url in df_urls_licitaciones:
#    print(url)

In [ ]:
# Generar llamada a API para obtener las licitaciones diarias desde 1/1/2025 al 10/01/2025
lista_fechas = generar_lista_fechas(1,1,2025,  2,1,2025)
df_urls_licitaciones = generar_url_licitaciones(lista_fechas)

df_licitaciones = pd.DataFrame()
for url in df_urls_licitaciones:
    df_aux = get_lista_licitaciones(url)
    df_licitaciones = pd.concat([df_licitaciones, df_aux], ignore_index=True)
    n=len(df_aux)
    print(n, "-", url)
    time.sleep(1.5)
df_licitaciones.head(5)

In [ ]:
df_licitaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376 entries, 0 to 375
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Cantidad       376 non-null    int64 
 1   FechaCreacion  376 non-null    object
 2   Version        376 non-null    object
 3   Listado        376 non-null    object
dtypes: int64(1), object(3)
memory usage: 11.9+ KB


In [ ]:


# Expandir los datos dentro de 'Listado'
df_detalle_licitaciones = json_normalize(df_licitaciones['Listado'])

df_detalle_licitaciones.head()


,CodigoExterno,Nombre,CodigoEstado,FechaCierre
0,1288-43-LR24,Medios de cultivo y susceptibilidad y medios c...,6,2025-01-02T15:00:00
1,1514-189-LE24,Contratación de Prestación de Servicios para e...,6,2025-01-02T19:00:00
2,1704-21-LR24,SERVICIO DE SEGURIDAD PARA EL HOSPITAL BIPROVI...,6,2025-01-02T15:30:00
3,2069-221-LP24,YOE INSUMOS CLÍNICOS PARA ATENCIÓN CERRADA,6,2025-01-02T17:00:00
4,2207-23-LE24,ADQUISICION DE SERVICIO EN APOYO A LA GESTIÓN ...,6,2025-01-02T15:30:00


In [ ]:
df_detalle_licitaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7839 entries, 0 to 7838
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CodigoExterno  7839 non-null   object
 1   Nombre         7839 non-null   object
 2   CodigoEstado   7839 non-null   int64 
 3   FechaCierre    7839 non-null   object
dtypes: int64(1), object(3)
memory usage: 245.1+ KB


# OBTENER RESUMEN DE LAS LICITACIONES DEL LISTADO ANTERIOR

In [ ]:
url_detalle_ej="https://api.mercadopublico.cl/servicios/v1/publico/licitaciones.json?codigo=1288-43-LR24&ticket=8758C2C5-3863-4889-B05B-2D7195E78079"
response = requests.get(url_detalle_ej)
licitacion = json.loads(response.content.decode('utf-8'))  # Decode using UTF-8
licitacion

{'Cantidad': 1,
 'FechaCreacion': '2025-05-04T06:40:32.7821714Z',
 'Version': 'v1',
 'Listado': [{'CodigoExterno': '1288-43-LR24',
   'Nombre': 'Medios de cultivo y susceptibilidad y medios cromogénicos',
   'CodigoEstado': 6,
   'Descripcion': 'ADQUIRIR SUMINISTRO DE MEDIOS DE CULTIVO PRIMARIOS, SUSCEPTIBILIDAD Y MEDIOS DE CULTIVO CROMOGÉNICOS PARA LA REALIZACIÓN DE EXÁMENES MICROBIOLÓGICOS DESTINADOS A LOS ESTABLECIMIENTOS DEPENDIENTES DE LA RED DEL SERVICIO DE SALUD METROPOLITANO OCCIDENTE',
   'FechaCierre': None,
   'Estado': 'Cerrada',
   'Comprador': {'CodigoOrganismo': '7029',
    'NombreOrganismo': 'SERVICIO SALUD METROPOLITANO OCCIDENTE',
    'RutUnidad': '61.608.200-0',
    'CodigoUnidad': '2284',
    'NombreUnidad': 'Servicio de Salud Metropolitano Occidente',
    'DireccionUnidad': 'Libertador Bernardo O´Higgnis N° 2429, Santiago',
    'ComunaUnidad': 'Santiago',
    'RegionUnidad': 'Región Metropolitana de Santiago',
    'RutUsuario': '',
    'CodigoUsuario': '1934808',
 

In [ ]:
l2 = json_normalize(licitacion['Listado'])
l2

,CodigoExterno,Nombre,CodigoEstado,Descripcion,FechaCierre,Estado,DiasCierreLicitacion,Informada,CodigoTipo,Tipo,...,Fechas.FechaAdjudicacion,Fechas.FechaEstimadaAdjudicacion,Fechas.FechaSoporteFisico,Fechas.FechaTiempoEvaluacion,Fechas.FechaEstimadaFirma,Fechas.FechasUsuario,Fechas.FechaVisitaTerreno,Fechas.FechaEntregaAntecedentes,Items.Cantidad,Items.Listado
0,1288-43-LR24,Medios de cultivo y susceptibilidad y medios c...,6,ADQUIRIR SUMINISTRO DE MEDIOS DE CULTIVO PRIMA...,None,Cerrada,0,0,1,LR,...,2025-04-24T19:00:00,2025-04-24T19:00:00,None,None,None,None,None,None,1,"[{'Correlativo': 1, 'CodigoProducto': 41104308..."


In [ ]:
print(l2["Descripcion"][0])

ADQUIRIR SUMINISTRO DE MEDIOS DE CULTIVO PRIMARIOS, SUSCEPTIBILIDAD Y MEDIOS DE CULTIVO CROMOGÉNICOS PARA LA REALIZACIÓN DE EXÁMENES MICROBIOLÓGICOS DESTINADOS A LOS ESTABLECIMIENTOS DEPENDIENTES DE LA RED DEL SERVICIO DE SALUD METROPOLITANO OCCIDENTE


In [ ]:
print(l2["Items.Listado"][0])
l3 = json_normalize(l2["Items.Listado"][0])
l3

[{'Correlativo': 1, 'CodigoProducto': 41104308, 'CodigoCategoria': '41104300', 'Categoria': 'Equipamiento para laboratorios / Equipos e insumos para laboratorio / Equipo de fermentación', 'NombreProducto': 'Sistemas de cultivo ambiental anaeróbico o suministros', 'Descripcion': 'SUMINISTRO DE MEDIOS DE CULTIVO PRIMARIOS, SUSCEPTIBILIDAD Y MEDIOS DE CULTIVO CROMOGÉNICOS PARA LA REALIZACIÓN DE EXÁMENES MICROBIOLÓGICOS DESTINADOS A LOS ESTABLECIMIENTOS DEPENDIENTES DE LA RED DEL SERVICIO DE SALUD METROPOLITANO OCCIDENTE', 'UnidadMedida': 'Unidad', 'Cantidad': 1.0, 'Adjudicacion': None}]


,Correlativo,CodigoProducto,CodigoCategoria,Categoria,NombreProducto,Descripcion,UnidadMedida,Cantidad,Adjudicacion
0,1,41104308,41104300,Equipamiento para laboratorios / Equipos e ins...,Sistemas de cultivo ambiental anaeróbico o sum...,"SUMINISTRO DE MEDIOS DE CULTIVO PRIMARIOS, SUS...",Unidad,1.0,None


In [ ]:
# agregar columnas a un dataframe
df_detalle_licitaciones["Descripcion"]=None
df_detalle_licitaciones["CodigoCategoria"]=None
df_detalle_licitaciones["Categoria"]=None
df_detalle_licitaciones["CodigoProducto"]=None
df_detalle_licitaciones["NombreProducto"]=None

df_detalle_licitaciones["CodigoOrganismo"]=None
df_detalle_licitaciones["NombreOrganismo"]=None
df_detalle_licitaciones["RutUnidad"]=None
df_detalle_licitaciones["ComunaUnidad"]=None
df_detalle_licitaciones["RegionUnidad"]=None



In [ ]:
df_detalle_licitaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747 entries, 0 to 746
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CodigoExterno    747 non-null    object
 1   Nombre           747 non-null    object
 2   CodigoEstado     747 non-null    int64 
 3   FechaCierre      747 non-null    object
 4   Descripcion      0 non-null      object
 5   CodigoCategoria  0 non-null      object
 6   Categoria        0 non-null      object
 7   CodigoProducto   0 non-null      object
 8   NombreProducto   0 non-null      object
 9   CodigoOrganismo  0 non-null      object
 10  NombreOrganismo  0 non-null      object
 11  RutUnidad        0 non-null      object
 12  ComunaUnidad     0 non-null      object
 13  RegionUnidad     0 non-null      object
dtypes: int64(1), object(13)
memory usage: 81.8+ KB


In [ ]:
 df_detalle_licitaciones = completar_detalle(df_detalle_licitaciones,0,10)
 df_detalle_licitaciones

KeyboardInterrupt: 

# LAnzo todo el proceso para que corra solo

In [ ]:
# Generar llamada a API para obtener las licitaciones diarias desde 1/1/2025 al 10/01/2025
lista_fechas = generar_lista_fechas(1,1,2025,  4,5,2025)
df_urls_licitaciones = generar_url_licitaciones(lista_fechas)

df_licitaciones = pd.DataFrame()
for url in df_urls_licitaciones:
    df_aux = get_lista_licitaciones(url)
    df_licitaciones = pd.concat([df_licitaciones, df_aux], ignore_index=True)
    time.sleep(1.5)
n=len(df_licitaciones )
print("CAntidad de licitaciones", n)

# Expandir los datos dentro de 'Listado'
df_detalle_licitaciones = json_normalize(df_licitaciones['Listado'])

# agregar columnas a un dataframe
df_detalle_licitaciones["Descripcion"]=None
df_detalle_licitaciones["CodigoCategoria"]=None
df_detalle_licitaciones["Categoria"]=None
df_detalle_licitaciones["CodigoProducto"]=None
df_detalle_licitaciones["NombreProducto"]=None

df_detalle_licitaciones["CodigoOrganismo"]=None
df_detalle_licitaciones["NombreOrganismo"]=None
df_detalle_licitaciones["RutUnidad"]=None
df_detalle_licitaciones["ComunaUnidad"]=None
df_detalle_licitaciones["RegionUnidad"]=None

df_detalle_licitaciones = completar_detalle(df_detalle_licitaciones)

# Retomar desde un archivo

In [9]:
# leer archivo csv
id_retoma = "16823_base1"
myfile = MydirData + 'df_licitaciones_' + id_retoma + '.xlsx'
print(myfile)
#df_detalle_licitaciones = pd.read_csv(myfile, encoding="utf-8")

df_detalle_licitaciones = pd.read_excel(myfile)

df_detalle_licitaciones.info()


/content/drive/MyDrive/UDD_Data_Science/CasoUso/data/df_licitaciones_16823_base1.xlsx
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47492 entries, 0 to 47491
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CodigoExterno    47492 non-null  object 
 1   Nombre           47492 non-null  object 
 2   CodigoEstado     47492 non-null  int64  
 3   FechaCierre      47482 non-null  object 
 4   Descripcion      16823 non-null  object 
 5   CodigoCategoria  16823 non-null  float64
 6   Categoria        16823 non-null  object 
 7   CodigoProducto   16823 non-null  float64
 8   NombreProducto   16822 non-null  object 
 9   CodigoOrganismo  16823 non-null  float64
 10  NombreOrganismo  16823 non-null  object 
 11  RutUnidad        16823 non-null  object 
 12  ComunaUnidad     15585 non-null  object 
 13  RegionUnidad     16823 non-null  object 
 14  go               545 non-null    float64
 15  orden            4

In [11]:
i=16823-1
df_detalle_licitaciones.iloc[i]

,16822
CodigoExterno,621-868-LR24
Nombre,LATANOPROST 50MCGML S.OF FRA 25 A 3 ML 1000020377
CodigoEstado,8
FechaCierre,2024-11-28 16:00:00
Descripcion,"\nBASES ADMINISTRATIVAS Y TÉCNICAS TIPO, POR L..."
CodigoCategoria,51241100.0
Categoria,Medicamentos y productos farmacéuticos / Medic...
CodigoProducto,51241110.0
NombreProducto,Latanoprost
CodigoOrganismo,6957.0


In [21]:
df_detalle_licitaciones = completar_detalle(df_detalle_licitaciones,-1,i)



stop: -1    i= 16822
16822  -  621-868-LR24
https://api.mercadopublico.cl/servicios/v1/publico/licitaciones.json?codigo=621-868-LR24&ticket=8758C2C5-3863-4889-B05B-2D7195E78079
---GetDescripcion-----


KeyboardInterrupt: 